# BGG Da2 Score 

The objetive is the get all Da2 guild user games Score 

### Import Libaries 

In [115]:
import requests as rq
import xml.etree.ElementTree as ET
import time 
import json
import pandas as pd

### Get Da2 Guilds Users

In [68]:
uri = 'http://www.boardgamegeek.com/xmlapi2/guild?id=862&members=1'
da2_guild = rq.get(uri)
while (da2_guild.status_code == 202):
    time.sleep(10)
    da2_guild = rq.get(uri)

In [69]:
# Check bgg results 
print(da2_guild.status_code )

200


In [70]:
# Parse Bgg 
da2_guild_parse = ET.fromstring(da2_guild.text )

In [71]:
# Calculate Guild members pages 

guild_members_num = int(da2_guild_parse.find('members').get('count'))

num_pages = round(guild_members_num/25) + 1


In [76]:
# Get Da2 guild Members list 

da2_guild_members = list()

for page in range(1,num_pages+1):
    uri = 'http://www.boardgamegeek.com/xmlapi2/guild?id=862&members=1&page='+str(page)
    da2_guild = rq.get(uri)
    while (da2_guild.status_code == 202):
        time.sleep(50)
        da2_guild = rq.get(uri)        
    da2_guild_parse = ET.fromstring(da2_guild.text )    
    for i in da2_guild_parse.find('members'):
        da2_guild_members.append(i.get('name'))

    

    

['5Renton', 'abendoso', 'Aecios', 'allaro1', 'Anaskela', 'angeru', 'atlias', 'atorija', 'Awell', 'ChrisBlair', 'cptbucci', 'D0NK1J0T3', 'dariodorado', 'des06', 'eduvigis', 'estheon', 'finijorge', 'freakowsky', 'Greypilgrim', 'gvt2000', 'igores', 'Ikarus', 'itus', 'jasorel', 'jsper', 'kalimatias', 'kastel', 'kebrantador', 'Koblar', 'LADER', 'Lochi', 'lostrikis', 'malarrama', 'Manu Chao', 'marantito74', 'Michel13', 'muyoze', 'netskaven', 'olaki', 'pepelessar1', 'Peto', 'raxar', 'Razmad', 'Renton', 'rgdebarros', 'rkoko', 'RobCry', 'roblax', 'ryoga121', 'sandraynapi', 'schonochini', 'silmarwen754', 'Stoicakovic', 'Tache', 'taiho', 'terKo', 'Urban Noises', 'uriyum', 'vonkhades', 'Wizzy Parkerir', 'zorro_jv']


In [87]:
# Get User Game notes 



print(da2_guild_members)
print(da2_guild_members[0:3])



    

['5Renton', 'abendoso', 'Aecios', 'allaro1', 'Anaskela', 'angeru', 'atlias', 'atorija', 'Awell', 'ChrisBlair', 'cptbucci', 'D0NK1J0T3', 'dariodorado', 'des06', 'eduvigis', 'estheon', 'finijorge', 'freakowsky', 'Greypilgrim', 'gvt2000', 'igores', 'Ikarus', 'itus', 'jasorel', 'jsper', 'kalimatias', 'kastel', 'kebrantador', 'Koblar', 'LADER', 'Lochi', 'lostrikis', 'malarrama', 'Manu Chao', 'marantito74', 'Michel13', 'muyoze', 'netskaven', 'olaki', 'pepelessar1', 'Peto', 'raxar', 'Razmad', 'Renton', 'rgdebarros', 'rkoko', 'RobCry', 'roblax', 'ryoga121', 'sandraynapi', 'schonochini', 'silmarwen754', 'Stoicakovic', 'Tache', 'taiho', 'terKo', 'Urban Noises', 'uriyum', 'vonkhades', 'Wizzy Parkerir', 'zorro_jv']
['5Renton', 'abendoso', 'Aecios']


In [142]:
da2_user = {}


for i in da2_guild_members:

    uri = 'http://www.boardgamegeek.com/xmlapi2/collection?username='+str(i)+'&stats=1&rated=1'
    bgg_user = rq.get(uri)
    while (bgg_user.status_code == 202):
        time.sleep(10)
        bgg_user = rq.get(uri)
        
    user_col = ET.fromstring(bgg_user.text)
    
    notas_dic = {}
    
    for juego in user_col.findall('item'):
        #rating = juego.find("rating")
        rating = juego.find("stats").find("rating")
        ranks = rating.find("ranks")
        for r in ranks.findall("rank"):
            rank_str = r.get("value")
            if (rank_str != "Not Ranked" and r.get("name") == "boardgame"):
                if (int(rank_str) <= 10):
                    if(juego.find("originalname") == None ):
                        nombre = juego.find("name").text
                    else:
                        nombre = juego.find("originalname").text

                    nota = rating.get("value")
        #print(str(nombre) + ": " + str(nota))
                    notas_dic[nombre] = nota
        #print (rating.get("value"))
      
        
    da2_user[str(i)]=notas_dic


In [143]:
da2_user_json = json.JSONEncoder().encode(da2_user)

In [144]:
with open('C:\Temp\da2_users.json', 'w') as outfile:
    json.dump(da2_user_json, outfile)

In [145]:
outfile.close()